In [ ]:
# ============================================================
# Bibliotecas necessárias para análise de dados e visualização
# ============================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings  # importar o módulo warnings

warnings.filterwarnings('ignore')  # ignorar todos os warnings

# ============================================================
# ALÍNEA 1: Carregar o ficheiro CSV, verificar dimensão e sumário
# ============================================================

# 1. Carregar o dataset com separador e decimal corretos
df = pd.read_csv("C:/Users/E6103/Desktop/Anadi/ANADI_1221294_1211981_1200546/TRAB2/AIRPOL_data.csv", sep=";", decimal=",")

# 2. Remover colunas que foram criadas automaticamente e estão vazias (ex: "Unnamed")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

# 3. Contar e remover linhas duplicadas para evitar análises repetidas ou enviesadas
duplicados = df.duplicated().sum()
print(f"\nNúmero de linhas duplicadas encontradas: {duplicados}")
if duplicados > 0:
    df = df.drop_duplicates()
    print("Linhas duplicadas removidas.")

# 4. Mostrar o número de linhas e colunas do dataset
print("\nDimensões do dataset:", df.shape)

# 5. Mostrar as primeiras 5 linhas para entender como os dados estão estruturados
print("\nPrimeiras 5 linhas dos dados:")
print(df.head())

# 6. Estatísticas descritivas para todas as colunas (numéricas e categóricas)
print("\nResumo estatístico (antes da conversão):")
print(df.describe(include='all'))

# 7. Verificar tipos de dados e quantidade de valores nulos por coluna
print("\nInformação dos dados:")
print(df.info())

print("\nValores nulos por coluna:")
print(df.isnull().sum())


In [ ]:
# ============================================================
# ALÍNEA 2 — Gráficos para análise exploratória dos dados
# ============================================================

# Definir estilo visual para gráficos e tamanho padrão das figuras
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# ----------------------------
# Variáveis numéricas
# ----------------------------

# Lista das colunas que possuem dados numéricos para análise
numerical_cols = [
    'Affected_Population',
    'Populated_Area[km2]',
    'Air_Pollution_Average[ug/m3]',
    'Value'  # Coluna ainda com nome original nesta fase
]

# Para cada coluna numérica, plotar histograma + KDE e boxplot
for col in numerical_cols:
    plt.figure(figsize=(14, 5))

    # Histograma + curva de densidade (KDE) para mostrar distribuição dos dados
    plt.subplot(1, 2, 1)
    sns.histplot(df[col], kde=True, bins=30, color="skyblue")
    plt.title(f'Histograma de {col}')

    # Boxplot para identificar outliers e ver resumo estatístico visualmente
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df[col], color="lightcoral")
    plt.title(f'Boxplot de {col}')

    # Ajustar layout para evitar sobreposição dos gráficos
    plt.tight_layout()
    plt.show()

# ----------------------------
# Variáveis categóricas
# ----------------------------

# Lista das colunas categóricas para análise de frequência
categorical_cols = ['Country', 'NUTS_Code', 'Air_Pollutant', 'Outcome']

# Para cada coluna categórica, plotar gráfico de barras (contagem)
for col in categorical_cols:
    n_uniques = df[col].nunique()  # Quantidade de categorias diferentes

    # Ajustar tamanho da figura conforme número de categorias (mais categorias = figura maior)
    plt.figure(figsize=(12, 6 if n_uniques <= 15 else 10))

    if n_uniques > 20:
        # Se muitas categorias, mostrar somente as 20 mais frequentes para melhor visualização
        top_values = df[col].value_counts().nlargest(20).index
        df_top = df[df[col].isin(top_values)]
        sns.countplot(data=df_top, y=col, order=top_values, palette="viridis")
        plt.title(f"Top 20 categorias em {col}")
    else:
        # Se poucas categorias, mostrar todas
        sns.countplot(data=df, y=col, order=df[col].value_counts().index, palette="viridis")
        plt.title(f"Distribuição de {col}")

    # Legendas dos eixos
    plt.xlabel("Contagem")
    plt.ylabel(col)

    # Ajuste do layout para visualização correta
    plt.tight_layout()
    plt.show()


In [ ]:
# ============================================================
# ALÍNEA 4 — Agrupamento dos países por região
# ============================================================

# Renomear coluna 'Value' para 'Premature_Deaths'
df.rename(columns={'Value': 'Premature_Deaths'}, inplace=True)

# Atualizar array de colunas numéricas para refletir o novo nome da coluna
numerical_cols = ['Premature_Deaths' if col == 'Value' else col for col in numerical_cols]

western = ['Austria', 'Belgium', 'France', 'Germany', 'Netherlands', 'Switzerland']
eastern = ['Poland', 'Czechia', 'Hungary']
southern = ['Greece', 'Spain', 'Italy', 'Portugal']
northern = ['Sweden', 'Denmark', 'Finland', 'Northern Europe']

# Função para mapear país à respectiva região
def map_region(country):
    if country in western:
        return 'Western Europe'
    elif country in eastern:
        return 'Eastern Europe'
    elif country in southern:
        return 'Southern Europe'
    elif country in northern:
        return 'Northern Europe'
    else:
        return 'Other'

# Aplicar o mapeamento para criar a coluna 'Region'
df['Region'] = df['Country'].apply(map_region)

# Mostrar a contagem de registros por região
# 'Número de Registos' significa a quantidade de linhas (observações) do dataset para cada região
print("\nDistribuição por Região:")
print(df['Region'].value_counts())

# Gráfico da distribuição por região (quantidade de observações por região)
sns.countplot(data=df, x='Region', order=df['Region'].value_counts().index, palette='muted')
plt.title("Distribuição de Registos por Região")
plt.xlabel("Região")
plt.ylabel("Número de Registos")  # Número de observações/linhas do dataset em cada região
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()
